# Fetch Google travel times

In [1]:
import urllib2, json
import pandas as pd
import numpy as np
import datetime, time
from SECRET_KEY import secret_key

In [2]:
# Load TAZ records with x and y coordinates attached
taz = pd.read_csv(r'..\data\taz_xy.txt')

meter_mile_conversion = 0.000621371

- define extraction time (for future travel time predictions)
- set skim as auto

In [3]:
# Create a Google-formatted coordinates field
taz['g_coord'] = taz['y_gps'].astype('str') + ',' + taz['x_gps'].astype('str')

# Look up series of travel times for a single origin

In [4]:
def fetch_times(otaz_list, dtaz_list, dep_time, traffic_model='best_guess'):
    results = {}
    # Look up 
    for otaz in otaz_list:
        print 'fetching times from zone: ' + str(otaz)

        urlfeed = ""

        origin = taz[taz['ID'] == otaz]['g_coord'].values[0]
        # get list of different destinations
        destination = ''

        for dtaz in dtaz_list:
            if otaz != dtaz:    # skip intrazonal trips where otaz==dtaz
                destination += taz[taz['ID'] == dtaz]['g_coord'].values[0] + '|'
    #             dtaz_list.append(dtaz)
            # remove trailing |
        destination = destination[:-1]

        urlfeed += "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+origin+"&destinations="+destination+ \
                "&departure_time="+dep_time+'&traffic_model='+traffic_model+"&key="+secret_key+"&units=imperial"

        # Fetch url and store
        results[otaz] = json.loads(urllib2.urlopen(urlfeed).read())
        results['traffic_model']=traffic_model
        
    return results

In [5]:
# am_results['model'] = 

In [48]:
def clean_results(results):
    """
    Convert from json to pandas dataframe
    """
    results_list = []
    for origin in otaz_list:
        i = 0
        for dest in results[origin]['destination_addresses']:
            print dest
            results_dict = {}
            results_dict['origin'] = results[origin]['origin_addresses'][0]
            results_dict['destination'] = dest
            results_dict['distance'] = results[origin]['rows'][0]['elements'][i]['distance']['text']
            results_dict['time_ff'] = results[origin]['rows'][0]['elements'][i]['duration']['value']/60.0
            try:
                results_dict['time_congested'] = results[origin]['rows'][0]['elements'][i]['duration_in_traffic']['value']/60.0
            except:
                print 'mistake'
                results_dict['time_congested'] = -99
            i+=1
            results_list.append(results_dict)
    
    return pd.DataFrame(results_list)

In [7]:
otaz_list = [532,1552,2570,2286,3108,3625,3528,1653,1178,1354,973]
dtaz_list = otaz_list

# set standard departure time for tomorrow at 8 AM
dep_hr = 8
dep_time = datetime.datetime.now()

# October 18, current year
dep_time = dep_time.replace(hour=dep_hr,day=18,month=10)
dep_time = str(int(time.mktime(dep_time.timetuple())))

# Best Guess
am_results = fetch_times(otaz_list, dtaz_list, dep_time, traffic_model='best_guess')
df_am_best = clean_results(am_results)
df_am_best['model'] = am_results['traffic_model']

fetching times from zone: 532
fetching times from zone: 1552
fetching times from zone: 2570
fetching times from zone: 2286
fetching times from zone: 3108
fetching times from zone: 3625
fetching times from zone: 3528
fetching times from zone: 1653
fetching times from zone: 1178
fetching times from zone: 1354
fetching times from zone: 973


In [8]:
# Pessimistic
am_results_slow = fetch_times(otaz_list, dtaz_list, dep_time, traffic_model='pessimistic')
df_am_slow = clean_results(am_results_slow)
df_am_slow['model'] = am_results_slow['traffic_model']

fetching times from zone: 532
fetching times from zone: 1552
fetching times from zone: 2570
fetching times from zone: 2286
fetching times from zone: 3108
fetching times from zone: 3625
fetching times from zone: 3528
fetching times from zone: 1653
fetching times from zone: 1178
fetching times from zone: 1354
fetching times from zone: 973


In [51]:
# Optimistic
am_results_fast = fetch_times(otaz_list, dtaz_list, dep_time, traffic_model='optimistic')
df_am_fast = clean_results(am_results_fast)
df_am_fast['model'] = am_results_fast['traffic_model']

NE 10th St, Bellevue, WA 98004, USA
3200 184th St SW, Lynnwood, WA 98037, USA
2349 Dakota St, Everett, WA 98201, USA
501-505 E 25th St, Tacoma, WA 98421, USA
250 4th St, Bremerton, WA 98337, USA
10315 Silverdale Way NW, Silverdale, WA 98383, USA
NE 73rd Way, Redmond, WA 98052, USA
235-247 A St SE, Auburn, WA 98002, USA
116 Williams Ave S, Renton, WA 98057, USA
977 Southcenter Mall, Tukwila, WA 98188, USA
505 Madison St, Seattle, WA 98104, USA
3200 184th St SW, Lynnwood, WA 98037, USA
2349 Dakota St, Everett, WA 98201, USA
501-505 E 25th St, Tacoma, WA 98421, USA
mistake
250 4th St, Bremerton, WA 98337, USA
10315 Silverdale Way NW, Silverdale, WA 98383, USA
NE 73rd Way, Redmond, WA 98052, USA
235-247 A St SE, Auburn, WA 98002, USA
116 Williams Ave S, Renton, WA 98057, USA
977 Southcenter Mall, Tukwila, WA 98188, USA
505 Madison St, Seattle, WA 98104, USA
NE 10th St, Bellevue, WA 98004, USA
2349 Dakota St, Everett, WA 98201, USA
501-505 E 25th St, Tacoma, WA 98421, USA
250 4th St, Bremer

In [52]:
df_am = df_am_best.append(df_am_slow)
df_am = df_am.append(df_am_fast)
df_am = pd.pivot_table(df_am, index=['origin','destination'],columns='model',values='time_congested').reset_index()

In [11]:
# PM Peak period
dep_hr = 17
dep_time = datetime.datetime.now()

# October 18, current year
dep_time = dep_time.replace(hour=dep_hr,day=18,month=10)
dep_time = str(int(time.mktime(dep_time.timetuple())))

# Best Guess
pm_results = fetch_times(otaz_list, dtaz_list, dep_time, traffic_model='best_guess')
df_pm_best = clean_results(pm_results)
df_pm_best['model'] = am_results['traffic_model']

fetching times from zone: 532
fetching times from zone: 1552
fetching times from zone: 2570
fetching times from zone: 2286
fetching times from zone: 3108
fetching times from zone: 3625
fetching times from zone: 3528
fetching times from zone: 1653
fetching times from zone: 1178
fetching times from zone: 1354
fetching times from zone: 973


In [12]:
# Pessimistic
pm_results_slow = fetch_times(otaz_list, dtaz_list, dep_time, traffic_model='pessimistic')
df_pm_slow = clean_results(pm_results_slow)
df_pm_slow['model'] = pm_results_slow['traffic_model']

fetching times from zone: 532
fetching times from zone: 1552
fetching times from zone: 2570
fetching times from zone: 2286
fetching times from zone: 3108
fetching times from zone: 3625
fetching times from zone: 3528
fetching times from zone: 1653
fetching times from zone: 1178
fetching times from zone: 1354
fetching times from zone: 973


In [13]:
# Optimistic
pm_results_fast = fetch_times(otaz_list, dtaz_list, dep_time, traffic_model='optimistic')
df_pm_fast = clean_results(pm_results_fast)
df_pm_fast['model'] = pm_results_fast['traffic_model']

fetching times from zone: 532
fetching times from zone: 1552
fetching times from zone: 2570
fetching times from zone: 2286
fetching times from zone: 3108
fetching times from zone: 3625
fetching times from zone: 3528
fetching times from zone: 1653
fetching times from zone: 1178
fetching times from zone: 1354
fetching times from zone: 973


In [53]:
df_pm = df_pm_best.append(df_pm_slow)
df_pm = df_pm.append(df_pm_fast)
df_pm = pd.pivot_table(df_pm, index=['origin','destination'],columns='model',values='time_congested').reset_index()

In [54]:
df_am['time'] = 'am'
df_pm['time'] = 'pm'
df = df_am.append(df_pm)
df['date'] = '10/18/2017'

In [55]:
df['d_city'] = df['destination'].apply(lambda row: row.split(',')[1].strip())
df['o_city'] = df['origin'].apply(lambda row: row.split(',')[1].strip())

In [56]:
# Look up TAZ
taz_city = {'Seattle':532,
           'Bellevue':1552,
           'Lynnwood':2570,
           'Everett':2286,
           'Tacoma':3108,
           'Bremerton':3625,
           'Silverdale':3528,
           'Redmond':1653,
           'Auburn':1178,
           'Renton':1354,
           'Tukwila':973}

df['dtaz'] = df['d_city'].apply(lambda row: taz_city[row])
df['otaz'] = df['o_city'].apply(lambda row: taz_city[row])

In [58]:
df.to_csv(r'A:\brice\soundcast_2014\scripts\summarize\inputs\network_summary\od_travel_times.csv',index=False)

In [59]:
df

model,origin,destination,best_guess,optimistic,pessimistic,time,date,d_city,o_city,dtaz,otaz
0,"10315 Silverdale Way NW, Silverdale, WA 98383,...","116 Williams Ave S, Renton, WA 98057, USA",79.333333,70.416667,97.083333,am,10/18/2017,Renton,Silverdale,1354,3528
1,"10315 Silverdale Way NW, Silverdale, WA 98383,...","2349 Dakota St, Everett, WA 98201, USA",115.550000,108.183333,127.133333,am,10/18/2017,Everett,Silverdale,2286,3528
2,"10315 Silverdale Way NW, Silverdale, WA 98383,...","235-247 A St SE, Auburn, WA 98002, USA",60.483333,56.000000,70.433333,am,10/18/2017,Auburn,Silverdale,1178,3528
3,"10315 Silverdale Way NW, Silverdale, WA 98383,...","250 4th St, Bremerton, WA 98337, USA",21.300000,18.550000,25.816667,am,10/18/2017,Bremerton,Silverdale,3625,3528
4,"10315 Silverdale Way NW, Silverdale, WA 98383,...","3200 184th St SW, Lynnwood, WA 98037, USA",106.000000,98.266667,116.216667,am,10/18/2017,Lynnwood,Silverdale,2570,3528
5,"10315 Silverdale Way NW, Silverdale, WA 98383,...","501-505 E 25th St, Tacoma, WA 98421, USA",46.983333,43.533333,54.250000,am,10/18/2017,Tacoma,Silverdale,3108,3528
6,"10315 Silverdale Way NW, Silverdale, WA 98383,...","505 Madison St, Seattle, WA 98104, USA",93.950000,80.966667,119.700000,am,10/18/2017,Seattle,Silverdale,532,3528
7,"10315 Silverdale Way NW, Silverdale, WA 98383,...","977 Southcenter Mall, Tukwila, WA 98188, USA",70.200000,64.583333,80.533333,am,10/18/2017,Tukwila,Silverdale,973,3528
8,"10315 Silverdale Way NW, Silverdale, WA 98383,...","NE 10th St, Bellevue, WA 98004, USA",97.766667,82.966667,123.500000,am,10/18/2017,Bellevue,Silverdale,1552,3528
9,"10315 Silverdale Way NW, Silverdale, WA 98383,...","NE 73rd Way, Redmond, WA 98052, USA",108.350000,91.783333,134.633333,am,10/18/2017,Redmond,Silverdale,1653,3528
